# ELM + Kmeans 

In [5]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin

# -------------------------
# HPELM Kullanılarak Scikit-Learn Uyumlu ELMRegressor Wrapper'ı
# -------------------------
from hpelm import ELM as HPELM

class HPELMRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_hidden=50, activation='sigmoid', random_state=42):
        self.n_hidden = n_hidden
        self.activation = activation
        self.random_state = random_state

    def fit(self, X, y):
        # Pandas DataFrame veya Series ise numpy array'e dönüştür
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values
        
        n_features = X.shape[1]
        # y eğer tek boyutlu ise n_out = 1, aksi halde y.shape[1] olarak ayarlanıyor.
        n_out = 1 if y.ndim == 1 else y.shape[1]
        # Eğer random_state verilmişse, NumPy random seed'i ayarlanıyor.
        if self.random_state is not None:
            np.random.seed(self.random_state)
        # HPELM modelini oluşturuyoruz; random_state parametresi kullanılmıyor.
        self.model_ = HPELM(n_features, n_out, classification='regression')
        # Seçilen aktivasyon fonksiyonuna göre n_hidden nöron ekleniyor.
        self.model_.add_neurons(self.n_hidden, self.activation)
        # Model eğitim verileri ile eğitiliyor. 'r' parametresi regresyon için kullanılır.
        self.model_.train(X, y, 'r')
        return self

    def predict(self, X):
        if hasattr(X, 'values'):
            X = X.values
        return self.model_.predict(X)

# -------------------------
# Ön İşlem ve Veri Hazırlığı
# -------------------------
# Veri setini yükleyip gereksiz sütunları kaldırıyoruz.
df = pd.read_excel("dataset.xlsx")
df.drop(columns=["Seli", "Tarih"], inplace=True, errors='ignore')

# mccinfo sütunundan dört haneli değeri çekmek için fonksiyon
def extract_mcc(value):
    if pd.isnull(value):
        return np.nan
    match = re.match(r"(\d{4})-", value)
    if match:
        return match.group(1)
    else:
        return np.nan

df['mccinfo'] = df['mccinfo'].apply(extract_mcc)
num_missing_mcc = df['mccinfo'].isnull().sum()
print(f"mccinfo değeri bulunamadığı için silinecek satır sayısı: {num_missing_mcc}")
df = df.dropna(subset=['mccinfo'])

# Sayısal sütunlardaki değerlerin düzenlenmesi
numeric_columns = [
    "bıst100_Kapanış", "bıst100_Açılış", "bıst100_Yüksek", "bıst100_Düşük",
    "Euro_Kapanış", "Euro_Açılış", "Euro_Yüksek", "Euro_Düşük",
    "USD_Kapanış", "USD_Açılış", "USD_Yüksek", "USD_Düşük",
]

def fix_numeric_value(s):
    s = s.replace(',', '.')
    if s.count('.') > 1:
        second_dot = s.find('.', s.find('.') + 1)
        s = s[:second_dot]
    return s

for col in numeric_columns:
    df[col] = df[col].astype(str).apply(fix_numeric_value)
    df[col] = df[col].astype(float)

# Kategorik sütunların kodlanması
categorical_columns = ["Şehir", "İlçe", "mccinfo"]
le = LabelEncoder()
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

# Hedef ve özniteliklerin ayrılması
target = "Tek Çekim Komisyon Oranı (Güncel)"
X = df.drop(columns=[target])
y = df[target]

# -------------------------
# Ölçeklendirme (Scaler Uygulaması)
# -------------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

# -------------------------
# Kümeleme İşlemi (KMeans)
# -------------------------
n_clusters = 5  # Küme sayısı
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
# Küme bilgilerini ölçeklendirilmiş veri setine ekliyoruz.
X_scaled['Cluster'] = clusters

print("Kümeleme işlemi tamamlandı. Küme dağılımı:")
print(pd.Series(clusters).value_counts())

# -------------------------
# Küme Bazında Model Eğitimi, Tahmin ve Metrik Hesaplaması (HPELMRegressor, Feature Selection olmadan)
# -------------------------
all_predictions = pd.Series(index=y.index, dtype=float)
cluster_metrics = {}

for cluster in np.unique(clusters):
    print(f"\nCluster {cluster} için model eğitimi başlatılıyor...")
    # İlgili kümeye ait verileri seçip 'Cluster' sütununu çıkarıyoruz
    cluster_mask = X_scaled['Cluster'] == cluster
    X_cluster = X_scaled.loc[cluster_mask].drop(columns=['Cluster'])
    y_cluster = y.loc[cluster_mask]
    
    # Eğer örnek sayısı 10'dan azsa, CV kat sayısı örnek sayısına göre ayarlanıyor
    if len(X_cluster) < 10:
        print(f"Uyarı: Cluster {cluster} için örnek sayısı {len(X_cluster)}, CV kat sayısı örnek sayısına göre ayarlanıyor.")
        kf = KFold(n_splits=len(X_cluster), shuffle=True, random_state=123)
    else:
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Pipeline: sadece HPELMRegressor kullanılıyor (özellik seçimi olmadan)
    pipeline = Pipeline([
        ('elm', HPELMRegressor(n_hidden=50, activation='tanh', random_state=42))
    ])
    
    # Cross-validation ile tahminleri elde ediyoruz
    y_pred_cluster = cross_val_predict(pipeline, X_cluster, y_cluster, cv=kf)
    
    # Küme bazında performans metriklerini hesaplıyoruz
    mae = mean_absolute_error(y_cluster, y_pred_cluster)
    mape = mean_absolute_percentage_error(y_cluster, y_pred_cluster)
    r2 = r2_score(y_cluster, y_pred_cluster)
    
    cluster_metrics[cluster] = {'MAE': mae, 'MAPE': mape, 'R2': r2}
    print(f"Cluster {cluster} sonuçları:")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.4f}")
    print(f"R2: {r2:.4f}")
    
    # Bu kümedeki tahminleri orijinal indeksleriyle saklıyoruz
    all_predictions.loc[cluster_mask] = y_pred_cluster

print("\nTüm kümeler için model sonuçları:")
print(cluster_metrics)

# -------------------------
# Genel Performans Metriklerinin Hesaplanması
# -------------------------
overall_mae = mean_absolute_error(y, all_predictions)
overall_mape = mean_absolute_percentage_error(y, all_predictions)
overall_r2 = r2_score(y, all_predictions)

print("\nGenel model sonuçları:")
print(f"Genel MAE: {overall_mae:.4f}")
print(f"Genel MAPE: {overall_mape:.4f}")
print(f"Genel R2: {overall_r2:.4f}")

# -------------------------
# Gerçek ve Tahmin Edilen Değerlerin Görselleştirilmesi
# -------------------------
plt.figure(figsize=(14,7))
plt.scatter(np.arange(len(y)), y, label="Gerçek Değerler", marker="o")
plt.plot(np.arange(len(y)), all_predictions.sort_index(), label="Tahmin Edilen Değerler", marker="x", color="red", linestyle="-")
plt.xlabel("Örnek Index")
plt.ylabel("Değer")
plt.title("Gerçek ve Tahmin Edilen Değerler (Feature Selection olmadan, KMeans + HPELMRegressor)")
plt.legend()
plt.grid(True)
plt.show()


mccinfo değeri bulunamadığı için silinecek satır sayısı: 15
Kümeleme işlemi tamamlandı. Küme dağılımı:
1    521
0    391
3    361
2    301
4    118
Name: count, dtype: int64

Cluster 0 için model eğitimi başlatılıyor...
Cluster 0 sonuçları:
MAE: 0.5412
MAPE: 0.1224
R2: -1.5683


c:\Users\yagiz\anaconda3\envs\img\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=7.
  warnings.warn(


ValueError: setting an array element with a sequence.